# [CDAF] Atividade 3

## Nome: Thiago Pádua de Carvalho

## Matrícula: 2020007066

## Referências
- [1] https://figshare.com/collections/Soccer_match_event_dataset/4415000
- [2] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.spadl.wyscout.convert_to_actions.html
- [3] https://github.com/TomDecroos/matplotsoccer
- [4] https://soccermatics.readthedocs.io/en/latest/gallery/lesson1/plot_PlottingShots.html
- [5] https://soccermatics.readthedocs.io/en/latest/gallery/lesson1/plot_PlottingPasses.html
- [6] https://soccermatics.readthedocs.io/en/latest/gallery/lesson1/plot_PassNetworks.html

## Questão 1
- Baixe o dataset 'Wyscout Europa Top 5 2017/2018' em [1].
- Escolha uma partida e carregue os dados de eventos em um dataframe do pandas.
- Converta os dados de eventos para SPADL [2].

In [94]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotsoccer
import numpy as np
import socceraction.spadl
import re

from functools import partial
from scipy import stats

ModuleNotFoundError: No module named 'dbglog'

O jogo escolhido é entre Liverpool e Manchester City pela Premier League 2017/2018, ocorrido em 14/01/2018.

In [23]:
matches_england = pd.read_json('../data/matches/matches_England.json')
liv_city_df = matches_england[(matches_england['label'] == 'Liverpool - Manchester City, 4 - 3')]
liv_city_id = liv_city_df.iloc[0]['wyId']
liv_city_id

2499943

In [84]:
events_england = pd.read_json('../data/events/events_England.json')
liv_city_events = events_england[(events_england['matchId'] == liv_city_id)]

In [87]:
correct_columns = {'eventId':'type_id', 'subEventName':'subtype_name', 'playerId':'player_id',
              'matchId':'game_id', 'eventName':'type_name', 'teamId':'team_id',
              'eventSec':'milliseconds', 'subEventId':'subtype_id', 'id':'event_id'
              }

liv_city_events.rename(columns=correct_columns, inplace=True)
liv_city_events['period_id'] = pd.factorize(liv_city_events['matchPeriod'])[0]+1
liv_city_events['milliseconds'] = liv_city_events['milliseconds'] * 1000  # convert to milliseconds
liv_city_events

,type_id,subtype_name,tags,player_id,positions,game_id,type_name,team_id,matchPeriod,milliseconds,subtype_id,event_id,period_id
378660,8,Simple pass,[{'id': 1801}],15808,"[{'y': 49, 'x': 49}, {'y': 41, 'x': 31}]",2499943,Pass,1612,1H,2528.085,85,220933014,1
378661,8,Simple pass,[{'id': 1801}],25393,"[{'y': 41, 'x': 31}, {'y': 70, 'x': 34}]",2499943,Pass,1612,1H,6517.082,85,220933015,1
378662,8,Simple pass,[{'id': 1801}],14870,"[{'y': 70, 'x': 34}, {'y': 41, 'x': 28}]",2499943,Pass,1612,1H,10732.865,85,220933016,1
378663,8,Simple pass,[{'id': 1801}],25393,"[{'y': 41, 'x': 28}, {'y': 50, 'x': 7}]",2499943,Pass,1612,1H,14110.714,85,220933017,1
378664,8,Launch,[{'id': 1802}],15175,"[{'y': 50, 'x': 7}, {'y': 9, 'x': 61}]",2499943,Pass,1612,1H,16587.550,84,220933018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380531,3,Free Kick,[{'id': 1801}],15175,"[{'y': 42, 'x': 10}, {'y': 48, 'x': 67}]",2499943,Free Kick,1612,2H,2983986.610,31,220934883,2
380532,1,Air duel,"[{'id': 701}, {'id': 1802}]",15808,"[{'y': 48, 'x': 67}, {'y': 35, 'x': 73}]",2499943,Duel,1612,2H,2988693.400,10,220934885,2
380533,1,Air duel,"[{'id': 703}, {'id': 1801}]",105339,"[{'y': 52, 'x': 33}, {'y': 65, 'x': 27}]",2499943,Duel,1625,2H,2988734.839,10,220934931,2
380534,8,Launch,[{'id': 1802}],9380,"[{'y': 65, 'x': 27}, {'y': 55, 'x': 46}]",2499943,Pass,1625,2H,2990658.798,84,220934933,2


In [91]:
liv_city_events['subtype_name'].unique()

array(['Simple pass', 'Launch', '', 'Free Kick', 'Touch',
       'Ground attacking duel', 'Ground defending duel',
       'Ball out of the field', 'Throw in', 'Ground loose ball duel',
       'Clearance', 'High pass', 'Head pass', 'Air duel', 'Smart pass',
       'Shot', 'Foul', 'Save attempt', 'Acceleration', 'Cross',
       'Goal kick', 'Hand pass', 'Free kick cross',
       'Goalkeeper leaving line', 'Corner', 'Reflexes',
       'Out of game foul', 'Hand foul'], dtype=object)

In [88]:
liv_city_spadl = socceraction.spadl.wyscout.convert_to_actions(liv_city_events, 1612)
liv_city_spadl

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,original_event_id,bodypart_id,type_id,result_id,action_id
0,2499943,1,2.528085,1612,15808,51.45,34.68,32.55,40.12,220933014,0,0,1,0
1,2499943,1,6.517082,1612,25393,32.55,40.12,35.70,20.40,220933015,0,0,1,1
2,2499943,1,10.732865,1612,14870,35.70,20.40,29.40,40.12,220933016,0,0,1,2
3,2499943,1,14.110714,1612,25393,29.40,40.12,7.35,34.00,220933017,0,0,1,3
4,2499943,1,16.587550,1612,15175,7.35,34.00,64.05,61.88,220933018,0,0,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,2499943,2,2873.344410,1612,8319,23.10,58.48,23.10,58.48,220934874,0,8,1,1406
1407,2499943,2,2948.558595,1625,38021,23.10,64.60,10.50,44.20,220934929,0,3,0,1407
1408,2499943,2,2983.986610,1612,15175,10.50,39.44,70.35,35.36,220934883,0,4,1,1408
1409,2499943,2,2990.658798,1625,9380,76.65,44.20,56.70,37.40,220934933,0,0,0,1409


## Questão 2
- Visualize uma sequência de 5 ações da partida usando matplotsoccer.actions [3].

In [116]:
typename = socceraction.spadl.actiontypes_df()
bodyparts = socceraction.spadl.bodyparts_df()
results = socceraction.spadl.results_df()
merged = pd.merge(liv_city_spadl, typename, on='type_id')
merged1 = pd.merge(merged, bodyparts, on='bodypart_id')
final_match_events = pd.merge(merged1, results, on='result_id')
final_match_events = final_match_events.drop(['type_id', 'bodypart_id', 'result_id'], axis=1)
final_match_events = final_match_events.rename(columns={'type_name': 'type', 'bodypart_name':'bodypart', 'result_name':'result'})
final_match_events

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,original_event_id,action_id,type,bodypart,result
0,2499943,1,2.528085,1612,15808,51.45,34.68,32.55,40.1200,220933014,0,pass,foot,success
1,2499943,1,6.517082,1612,25393,32.55,40.12,35.70,20.4000,220933015,1,pass,foot,success
2,2499943,1,10.732865,1612,14870,35.70,20.40,29.40,40.1200,220933016,2,pass,foot,success
3,2499943,1,14.110714,1612,25393,29.40,40.12,7.35,34.0000,220933017,3,pass,foot,success
4,2499943,1,38.497027,1625,9380,86.10,50.32,92.40,30.6000,220933055,6,pass,foot,success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,2499943,1,1252.306226,1612,222220,34.65,68.00,67.20,47.6000,220933395,328,throw_in,other,fail
1407,2499943,1,1654.602843,1612,222220,13.65,68.00,36.75,48.2800,220933473,412,throw_in,other,fail
1408,2499943,1,1840.148308,1612,257899,79.80,0.00,76.65,6.1200,220933543,454,throw_in,other,fail
1409,2499943,1,1856.054727,1612,15808,97.65,23.12,105.00,27.2000,220933559,464,shot,head/other,fail


In [120]:
# sort dataframe by period id and time when the event happened
final_match_events = final_match_events.sort_values(by=['period_id', 'time_seconds'])
final_match_events.reset_index(drop=True, inplace=True)
final_match_events

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,original_event_id,action_id,type,bodypart,result
0,2499943,1,2.528085,1612,15808,51.45,34.68,32.55,40.12,220933014,0,pass,foot,success
1,2499943,1,6.517082,1612,25393,32.55,40.12,35.70,20.40,220933015,1,pass,foot,success
2,2499943,1,10.732865,1612,14870,35.70,20.40,29.40,40.12,220933016,2,pass,foot,success
3,2499943,1,14.110714,1612,25393,29.40,40.12,7.35,34.00,220933017,3,pass,foot,success
4,2499943,1,16.587550,1612,15175,7.35,34.00,64.05,61.88,220933018,4,pass,foot,offside
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,2499943,2,2873.344410,1612,8319,23.10,58.48,23.10,58.48,220934874,1406,foul,foot,success
1407,2499943,2,2948.558595,1625,38021,23.10,64.60,10.50,44.20,220934929,1407,freekick_crossed,foot,fail
1408,2499943,2,2983.986610,1612,15175,10.50,39.44,70.35,35.36,220934883,1408,freekick_short,foot,success
1409,2499943,2,2990.658798,1625,9380,76.65,44.20,56.70,37.40,220934933,1409,pass,foot,fail


In [121]:
# finding succesful shots
succ_shots = final_match_events[(final_match_events['type'] == 'shot') & (final_match_events['result'] == 'success')]
succ_shots

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,original_event_id,action_id,type,bodypart,result
163,2499943,1,528.586171,1612,7868,85.05,21.76,105.0,37.4,220933208,163,shot,foot,success
638,2499943,1,2403.544100,1625,245364,6.30,19.72,0.0,30.6,220933931,638,shot,foot,success
958,2499943,2,836.718735,1612,15808,93.45,44.88,105.0,30.6,220934280,958,shot,foot,success
978,2499943,2,963.928509,1612,25747,91.35,43.52,105.0,37.4,220934313,978,shot,foot,success
1055,2499943,2,1323.837327,1612,120353,70.35,44.88,105.0,34.0,220934410,1055,shot,foot,success
1283,2499943,2,2292.703245,1625,265673,7.35,43.52,0.0,37.4,220934795,1283,shot,foot,success
1390,2499943,2,2757.268603,1625,14808,10.50,40.12,0.0,37.4,220934911,1390,shot,foot,success


In [ ]:
matplotsoccer.actions(
    location=actions[["start_x", "start_y", "end_x", "end_y"]],
    action_type=actions.type_name,
    team=actions.team_name,
    result= actions.result_name == "success",
    label=actions[["time_seconds", "type_name", "player_name", "team_name"]],
    labeltitle=["time","actiontype","player","team"],
    zoom=False
)

## Questão 3
- Visualize os chutes da partida, desenvolvendo seu código em cima do dataframe SPADL. Faça um plot para cada time. Adapte de [4].
- Qual time as melhores chances da partida? Por quê?

## Questão 4
- Escolha um jogador da partida que você escolheu.
- Faça um heatmap de todas ações dele [3].
- Faça um heatmap de todas as ações ofensivas dele [3].
- Faça um heatmap de todas as ações defensivas dele [3].
- O que você pode inferir sobre o comportamento do jogador? O comportamento dele varia muito do ataque para a defesa?

## Questão 5
- Para o mesmo jogador, crie um mapa de passes com os passes que ele efetuou na partida, desenvolvendo seu código em cima do dataframe SPADL. Adapte de [5].
- O mapa de passes trouxe alguma informação nova sobre o jogador?

## Questão 6
- Crie uma rede de passes de cada uma das equipes, desenvolvendo seu código em cima do dataframe SPADL. Adapte de [6].
- O que você consegue inferir sobre a formação de cada equipe? Quais jogadores de cada equipe possuem o maior grau (tem maior soma do peso das arestas)?